In [56]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm


In [57]:
df = pd.read_csv("/Users/ren/Desktop/untitled folder 2/sperformance.csv")

In [58]:
df.head()

,StudyHours,ExtraParticipation,PreviousMarks,PapersPracticed,Participation,SleepingHours,PerformanceIndex
0,4.003186,4.628357,73.481438,2.261419,20.226098,58.120590,75.197369
1,4.199477,8.960365,67.094194,3.017842,20.718872,47.583635,75.950440
2,4.211207,13.310912,67.977125,2.613675,5.918198,52.167417,74.860714
3,4.834761,10.589920,70.889901,3.512929,20.217720,50.741590,77.542092
4,5.288463,7.179640,67.130999,2.878155,13.229082,48.989753,76.970957


In [59]:
df.ExtraParticipation.replace({"Yes": 1, "No": 0}, inplace=True)

/var/folders/_r/k50gsvyj2dj2hylwzztbbkjc0000gn/T/ipykernel_4479/3906994331.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [60]:
print(f"Tamaño filas: {len(df)}")
print(f"Tamaño columnas: {len(df.columns)}")

Tamaño filas: 1000
Tamaño columnas: 7


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   StudyHours          1000 non-null   float64
 1   ExtraParticipation  1000 non-null   float64
 2   PreviousMarks       1000 non-null   float64
 3   PapersPracticed     1000 non-null   float64
 4   Participation       1000 non-null   float64
 5   SleepingHours       1000 non-null   float64
 6   PerformanceIndex    1000 non-null   float64
dtypes: float64(7)
memory usage: 54.8 KB


In [62]:
print(df.isna().sum())

StudyHours            0
ExtraParticipation    0
PreviousMarks         0
PapersPracticed       0
Participation         0
SleepingHours         0
PerformanceIndex      0
dtype: int64


In [63]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(x=df["StudyHours"], nbinsx=20, name="Horas de estudio", opacity=0.7, marker_color="blue"))
fig.add_trace(go.Histogram(x=df["PerformanceIndex"], nbinsx=20, name="Índice de rendimiento", opacity=0.7, marker_color="orange"))
fig.add_trace(go.Histogram(x=df["PreviousMarks"], nbinsx=20, name="Calificaciones previas", opacity=0.7, marker_color="green"))
fig.add_trace(go.Histogram(x=df["SleepingHours"], nbinsx=20, name="Horas de sueño", opacity=0.7, marker_color="red"))
fig.add_trace(go.Histogram(x=df["PapersPracticed"], nbinsx=20, name="Exámenes practicados", opacity=0.7, marker_color="purple"))
fig.add_trace(go.Histogram(x=df["ExtraParticipation"], nbinsx=2, name="Participación extra", opacity=0.7, marker_color="brown"))
fig.update_layout(
    barmode="overlay",
    #xaxis_title="Horas de estudio y Participación extra",
    title="Histogramas de variables",
    yaxis_title="Frecuencia",
    legend_title="Variable",
    width=900,
    height=500
)
fig.show()

In [64]:
import plotly.express as px

# Calcula la matriz de correlación
corr_matrix = df.corr(numeric_only=True)

# Grafica la matriz de correlación como mapa de calor
fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu',
    title="Matriz de correlación"
)
fig.update_layout(width=700, height=600)
fig.show()

In [65]:
x1name = 'SleepingHours'
y1name = 'PerformanceIndex'
x1 = df[x1name].values
y1 = df[y1name].values
x1_mean = np.mean(x1)
y1_mean = np.mean(y1)
beta1_1 = np.sum((x1 - x1_mean) * (y1 - y1_mean)) / np.sum((x1 - x1_mean)**2)
beta1_0 = y1_mean - beta1_1 * x1_mean
print(f"beta0 = {beta1_0:.4f}")
print(f"beta1 = {beta1_1:.4f}")

beta0 = 80.0997
beta1 = -0.0121


In [66]:
y1_pred = beta1_0 + beta1_1 * x1

# graficar
fig = go.Figure()
fig.add_trace(go.Scatter(x=x1, y=y1, mode='markers', name='Datos'))
fig.add_trace(go.Scatter(x=x1, y=y1_pred, mode='lines', name='Regresión lineal'))
# Líneas de error (verticales)
for xi, yi, ypi in zip(x1, y1, y1_pred):
    fig.add_trace(go.Scatter(x=[xi, xi], y=[yi, ypi],mode="lines",line=dict(color="gray", width=1),showlegend=False))

fig.update_layout(title='Regresión lineal simple', xaxis_title=x1name, yaxis_title=y1name,width=800,height=600)
fig.show()

In [67]:
R1 = np.corrcoef(y1, y1_pred)[0, 1]
r2_1 = R1**2
print(f"R = {R1:.4f}")
print(f"R^2 = {r2_1:.4f}")

# RMSE (raíz del error cuadrático medio)
rmse1 = np.sqrt(mean_squared_error(y1, y1_pred))
print(f"RMSE = {rmse1:.4f}")

R = 0.0095
R^2 = 0.0001
RMSE = 4.9367


In [68]:
x2name = 'PreviousMarks'
y2name = 'PerformanceIndex'
x2 = df[x2name].values
y2 = df[y2name].values
x2_mean = np.mean(x2)
y2_mean = np.mean(y2)
beta2_1 = np.sum((x2 - x2_mean) * (y2 - y2_mean)) / np.sum((x2 - x2_mean)**2)
beta2_0 = y2_mean - beta2_1 * x2_mean
print(f"beta0 = {beta2_0:.4f}")
print(f"beta1 = {beta2_1:.4f}")

beta0 = 72.9650
beta1 = 0.0968


In [69]:
y2_pred = beta2_0 + beta2_1 * x2

# graficar
fig = go.Figure()
fig.add_trace(go.Scatter(x=x2, y=y2, mode='markers', name='Datos'))
fig.add_trace(go.Scatter(x=x2, y=y2_pred, mode='lines', name='Regresión lineal'))
# Líneas de error (verticales)
for xi, yi, ypi in zip(x2, y2, y2_pred):
    fig.add_trace(go.Scatter(x=[xi, xi], y=[yi, ypi],mode="lines",line=dict(color="gray", width=1),showlegend=False))

fig.update_layout(title='Regresión lineal simple', xaxis_title=x2name, yaxis_title=y2name,width=800,height=600)
fig.show()

In [70]:
R2 = np.corrcoef(y2, y2_pred)[0, 1]
r2_2 = R2**2
print(f"R = {R2:.4f}")
print(f"R^2 = {r2_2:.4f}")

# RMSE (raíz del error cuadrático medio)
rmse2 = np.sqrt(mean_squared_error(y2, y2_pred))
print(f"RMSE = {rmse2:.4f}")

R = 0.1494
R^2 = 0.0223
RMSE = 4.8815


In [71]:
x3name = 'StudyHours'
y3name = 'PerformanceIndex'
x3 = df[x3name].values
y3 = df[y3name].values
x3_mean = np.mean(x3)
y3_mean = np.mean(y3)
beta3_1 = np.sum((x3 - x3_mean) * (y3 - y3_mean)) / np.sum((x3 - x3_mean)**2)
beta3_0 = y3_mean - beta3_1 * x3_mean
print(f"beta0 = {beta3_0:.4f}")
print(f"beta1 = {beta3_1:.4f}")

beta0 = 69.8391
beta1 = 2.0217


In [72]:
y3_pred = beta3_0 + beta3_1 * x3

# graficar
fig = go.Figure()
fig.add_trace(go.Scatter(x=x3, y=y3, mode='markers', name='Datos'))
fig.add_trace(go.Scatter(x=x3, y=y3_pred, mode='lines', name='Regresión lineal'))
# Líneas de error (verticales)
for xi, yi, ypi in zip(x3, y3, y3_pred):
    fig.add_trace(go.Scatter(x=[xi, xi], y=[yi, ypi],mode="lines",line=dict(color="gray", width=1),showlegend=False))

fig.update_layout(title='Regresión lineal simple', xaxis_title=x3name, yaxis_title=y3name,width=800,height=600)
fig.show()

In [73]:
R3 = np.corrcoef(y3, y3_pred)[0, 1]
r2_3 = R3**2
print(f"R = {R3:.4f}")
print(f"R^2 = {r2_3:.4f}")

# RMSE (raíz del error cuadrático medio)
rmse = np.sqrt(mean_squared_error(y3, y3_pred))
print(f"RMSE = {rmse:.4f}")

R = 0.6212
R^2 = 0.3859
RMSE = 3.8688


In [74]:
# Modelo de regresion con tres variables independientes
X = df[['StudyHours', 'ExtraParticipation', 'PreviousMarks', 'SleepingHours', 'PapersPracticed', 'Participation']]
y = df['PerformanceIndex']
X_sm = sm.add_constant(X)
model = sm.OLS(y, X_sm).fit()

# Tabla con coeficientes y estadísticos
summary_df = pd.DataFrame({
    'Coeficiente': model.params,
    'Error estándar': model.bse,
    't-value': model.tvalues,
    'p-value': model.pvalues
})
print(summary_df)

# Mostrar en formato tabla interactiva
import plotly.figure_factory as ff
fig_table = ff.create_table(summary_df.round(4))
fig_table.update_layout(width=500)
fig_table.show()


                    Coeficiente  Error estándar    t-value        p-value
const                 64.201026        1.386455  46.305881  2.725876e-250
StudyHours             1.753571        0.074915  23.407374   7.614992e-97
ExtraParticipation    -0.030643        0.027242  -1.124840   2.609285e-01
PreviousMarks          0.040728        0.011565   3.521495   4.486272e-04
SleepingHours         -0.029289        0.020816  -1.407014   1.597360e-01
PapersPracticed        1.044022        0.194454   5.368983   9.860224e-08
Participation          0.120222        0.003449  34.857227  1.659231e-174


In [75]:
y_pred = model.predict(X_sm)

# R (coeficiente de correlación de Pearson)
R = np.corrcoef(y, y_pred)[0, 1]
r2_full = R**2
print(f"R = {R:.4f}")
print(f"R^2 = {r2_full:.4f}")

# RMSE (raíz del error cuadrático medio)

rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"RMSE = {rmse:.4f}")

R = 0.8621
R^2 = 0.7432
RMSE = 2.5017


In [76]:

# Centrar los datos
X = df[['StudyHours', 'ExtraParticipation', 'PreviousMarks', 'SleepingHours', 'PapersPracticed', 'Participation']]
X_centered = X - X.mean()

print("=== PCA en las variables predictoras ===")
pca_3d = PCA(n_components=3)
pca_3d.fit(X_centered)

print("Autovalores (varianzas explicadas):", pca_3d.explained_variance_)
print("Autovectores (componentes principales):\n", pca_3d.components_)
print("Varianza explicada (%):", pca_3d.explained_variance_ratio_ * 100)

# Cargas (pesos de cada variable en cada componente)
cargas = pd.DataFrame(pca_3d.components_.T, index=X.columns, columns=[f'PC{i+1}' for i in range(3)])
print("Cargas de las variables en cada componente principal:")
print(cargas)

=== PCA en las variables predictoras ===
Autovalores (varianzas explicadas): [540.89376734  57.80205021  15.01097346]
Autovectores (componentes principales):
 [[-3.86957741e-04 -3.85956980e-04  4.21086520e-02 -4.45786258e-03
   1.81583369e-03  9.99101293e-01]
 [-5.69658433e-03 -1.00505312e-01  9.92795587e-01 -4.56659517e-02
   1.90652686e-02 -4.21223265e-02]
 [ 1.46963932e-02  5.30838909e-02  5.09669809e-02  9.96910290e-01
   2.30897929e-02  2.28423884e-03]]
Varianza explicada (%): [86.57482378  9.25172855  2.40263886]
Cargas de las variables en cada componente principal:
                         PC1       PC2       PC3
StudyHours         -0.000387 -0.005697  0.014696
ExtraParticipation -0.000386 -0.100505  0.053084
PreviousMarks       0.042109  0.992796  0.050967
SleepingHours      -0.004458 -0.045666  0.996910
PapersPracticed     0.001816  0.019065  0.023090
Participation       0.999101 -0.042122  0.002284


In [77]:
explained_var = pca_3d.explained_variance_ratio_
cum_var = np.cumsum(explained_var)
components = np.arange(1, len(explained_var)+1)

# Gráfico Plotly
fig_var = go.Figure()

# Varianza explicada por componente
fig_var.add_trace(go.Bar(
    x=components,
    y=explained_var,
    name='Varianza individual',
    marker_color='royalblue'
))

# Varianza acumulada
fig_var.add_trace(go.Scatter(
    x=components,
    y=cum_var,
    name='Varianza acumulada',
    mode='lines+markers',
    line=dict(color='firebrick', width=3),
    marker=dict(size=8)
))

fig_var.update_layout(
    title='Varianza explicada por componente y varianza acumulada (X_noisy)',
    xaxis_title='Componente principal',
    yaxis_title='Fracción de varianza',
    yaxis=dict(range=[0,1.05]),
    legend=dict(x=0.7, y=0.75),
    width=500,
    height=400
)

fig_var.show()

In [78]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

# --- PCA con 3 componentes ---
X = df[['StudyHours', 'ExtraParticipation', 'PreviousMarks', 'SleepingHours', 'PapersPracticed', 'Participation']]
X_centered = X - X.mean()
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_centered)
y = df['PerformanceIndex'].values

# --- Modelo con 1 componente principal ---
X_pc1 = X_pca[:, [0]]
reg1 = LinearRegression().fit(X_pc1, y)
y_pred1 = reg1.predict(X_pc1)
R1 = np.corrcoef(y, y_pred1)[0, 1]
r2_pc1 = R1**2
rmse1 = np.sqrt(mean_squared_error(y, y_pred1))

# Gráfico 2D
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_pc1.flatten(), y=y, mode='markers', name='Datos'))
fig.add_trace(go.Scatter(x=X_pc1.flatten(), y=y_pred1, mode='lines', name='Regresión lineal'))
fig.update_layout(title='Performance Index vs PC1', xaxis_title='PC1', yaxis_title='Performance Index')
fig.show()

print(f"Modelo con 1 PC: R = {R1:.4f}, R^2 = {r2_pc1:.4f}, RMSE = {rmse1:.4f}")

# --- Modelo con 2 componentes principales ---
X_pc2 = X_pca[:, :2]
reg2 = LinearRegression().fit(X_pc2, y)
y_pred2 = reg2.predict(X_pc2)
R2 = np.corrcoef(y, y_pred2)[0, 1]
r2_pc2 = R2**2
rmse2 = np.sqrt(mean_squared_error(y, y_pred2))

# Gráfico 3D
fig3d = go.Figure()
fig3d.add_trace(go.Scatter3d(x=X_pc2[:,0], y=X_pc2[:,1], z=y, mode='markers', name='Datos'))
fig3d.add_trace(go.Scatter3d(x=X_pc2[:,0], y=X_pc2[:,1], z=y_pred2, mode='markers', name='Regresión lineal', marker=dict(color='red')))
fig3d.update_layout(title='Performance Index vs PC1 y PC2', scene=dict(
    xaxis_title='PC1', yaxis_title='PC2', zaxis_title='Performance Index'))
fig3d.show()

print(f"Modelo con 2 PCs: R = {R2:.4f}, R^2 = {r2_pc2:.4f}, RMSE = {rmse2:.4f}")

# --- Modelo con 3 componentes principales ---
X_pc3 = X_pca[:, :3]
reg3 = LinearRegression().fit(X_pc3, y)
y_pred3 = reg3.predict(X_pc3)
R3 = np.corrcoef(y, y_pred3)[0, 1]
r2_pc3 = R3**2
rmse3 = np.sqrt(mean_squared_error(y, y_pred3))

print(f"Modelo con 3 PCs: R = {R3:.4f}, R^2 = {r2_pc3:.4f}, RMSE = {rmse3:.4f}")

# --- Comparación ---
print("\nComparación de modelos:")
print(f"1 PC:  R = {R1:.4f}, R^2 = {r2_pc1:.4f}, RMSE = {rmse1:.4f}")
print(f"2 PCs: R = {R2:.4f}, R^2 = {r2_pc2:.4f}, RMSE = {rmse2:.4f}")
print(f"3 PCs: R = {R3:.4f}, R^2 = {r2_pc3:.4f}, RMSE = {rmse3:.4f}")

Modelo con 1 PC: R = 0.5800, R^2 = 0.3364, RMSE = 4.0216


Modelo con 2 PCs: R = 0.5851, R^2 = 0.3423, RMSE = 4.0038
Modelo con 3 PCs: R = 0.5853, R^2 = 0.3426, RMSE = 4.0029

Comparación de modelos:
1 PC:  R = 0.5800, R^2 = 0.3364, RMSE = 4.0216
2 PCs: R = 0.5851, R^2 = 0.3423, RMSE = 4.0038
3 PCs: R = 0.5853, R^2 = 0.3426, RMSE = 4.0029


In [79]:
# Modelo de regresión lineal múltiple con StudyHours y Participation
X_sel = df[['StudyHours', 'Participation']]
y_sel = df['PerformanceIndex']

# Agregar constante para el intercepto
X_sel_sm = sm.add_constant(X_sel)
model_sel = sm.OLS(y_sel, X_sel_sm).fit()

# Coeficientes
print("Coeficientes del modelo:")
print(model_sel.params)

# Predicciones
y_pred_sel = model_sel.predict(X_sel_sm)

# Métricas de desempeño
R_sel = np.corrcoef(y_sel, y_pred_sel)[0, 1]
r2_sel = R_sel**2
rmse_sel = np.sqrt(mean_squared_error(y_sel, y_pred_sel))
print(f"R = {R_sel:.4f}")
print(f"R^2 = {r2_sel:.4f}")
print(f"RMSE = {rmse_sel:.4f}")

Coeficientes del modelo:
const            66.687969
StudyHours        2.032205
Participation     0.123787
dtype: float64
R = 0.8515
R^2 = 0.7251
RMSE = 2.5886
